In [1]:
from kernels import * 
from learning_models import *
from tools import *
import pandas as pd
import numpy as np
from time import time 
from sklearn.svm import SVC
from autoreload import superreload

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
dataX = pd.read_csv("Data/Train/Xtr0.csv", header=None)

In [4]:
dataX.head()

,0
0,TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGG...
1,CGGAGTGATTCAGCAAGAAGGCTCAGACCATTTACACTTCTCCTCA...
2,GCCTCAGGCACTCTCCAGATGGTTAAGATATGGTGTAAGTGTATGT...
3,TCAGTCGCCTGGCCTTCCAGCCTCACAGGCAAGATAAGCCACCCTC...
4,CTTAATGGAAAAATTAGATTATGGGTTGGCTTACCTGGATTTACCT...


In [5]:
data = dataX[0].values.tolist()

In [6]:
data[:10]

['TCCTCAACTTTTATTGGGCCGCTGTGGCACCAGAATCTACGAATGGCGCCCTCTAGAGTTGTGTAAAGAAGTGGCGTCACCTCATTATAAATAAAAGGTTG',
 'CGGAGTGATTCAGCAAGAAGGCTCAGACCATTTACACTTCTCCTCACAGGGCAAAGACTTCTCAGAGACAACTAATATGGCCTAGTGCCTAGTCATTGGAC',
 'GCCTCAGGCACTCTCCAGATGGTTAAGATATGGTGTAAGTGTATGTGCGTGTCTGGCTGCAGTTGATAGTTGGGCAAAGAGGAAACTATCACAAAGAGGAA',
 'TCAGTCGCCTGGCCTTCCAGCCTCACAGGCAAGATAAGCCACCCTCACAGACTTCTCTGTGCTCCCATCTAAACATCTAAATCAGCCATCTGGTTTCCTTA',
 'CTTAATGGAAAAATTAGATTATGGGTTGGCTTACCTGGATTTACCTAGAGTGTCTGCCATGTCTATCAATTGGATTACGGGGCTTTTATTGATTAGGGGGA',
 'GAAACAAGCTAATATAAGGTGGGGGCCTGAAAATTTTATCCAATGGTCATTTAAGAATCGCAATCAATGTCCTCTAGATATATAATTCACCTAAATACGGA',
 'CTTTCTTATCTGTGTGGCTGTGGGCGTGTCTTTGGCAAGCCCCACTGTGAGAGTTCCCTTATCTATGCCTGCAGGCTGTTCTTTTATTTGAAAGAATTCAG',
 'TGATTACTGCATGGAAAAGGGGCCAGGGCAATTACTAGAGGGCTCAATCCCAGTAAGGGGATGTGTACCAGGGAGGGTTGCTTTGACAGGAGATCAGTCTA',
 'GAGATTCTTAACATGCAACCACAAATCGATGCAGTTATGGATCATGTTGAATGGCAACTCACCTGGCAGTGGAGAGCAGAGTTGTTGGCAATCACATATGA',
 'GGGGACTTCTTGAACTGAGCTCCTAGAGCAGCTGGGGTTGTGAT

In [7]:
d = []
for i in range(len(data)):
    s = data[i].replace("A", "0")
    s = s.replace("T", "1")
    s = s.replace("C", "2")
    s = s.replace("G", "3")
    d.append(list(map(int, s)))
data = d

In [8]:
del dataX

In [11]:
trie = MismatchTrie(verbose=0)
print("-------- Computing mismatch kernel --------")
start = time()
kern = trie.traverse(data, 4, 5, 1)[0]
#kern = normalize_kernel(kern)
end = time()
print("-------- Done in {:.3f} minutes --------".format((end-start)/60))

-------- Computing mismatch kernel --------
-------- Done in 90.561 minutes --------


In [13]:
kern.shape

(2000, 2000)

In [14]:
kern_data = pd.DataFrame(kern)

In [15]:
kern_data.to_csv('Kernel_MisMatch_4_1.csv')

In [ ]:
dataY.to_csv()

In [33]:
np.array(data).shape

(2000, 101)

In [16]:
dataY = pd.read_csv("Data/Train/Ytr0.csv", index_col=0)
y = dataY['Bound'].values
X = np.array(data)
K = kern

In [17]:
kern = normalize_kernel(kern)

In [54]:
X_train, X_test, y_train, y_test, K_train, K_test = train_test_split(X,y,kern,test_size=0.1)

Total number of examples : 2000
Ratio of positive samples : 0.50
Ratio of negative to positive labels in the data : 1.00


In [45]:
X_train, X_test, y_train, y_test, K_train, K_test = K_train_test_split(X,y,kern,test_size=0.2)

In [55]:
res = []
for lmdb in [0.001,0.01,0.05,0.1,1,10,100,1000]:
    model = SVM(lmbd=lmdb)
    model.train(K_train, y_train)
    y_train_pred = model.predict(K_train)
    y_test_pred = model.predict(K_test)
    train_score = accuracy_score(y_train, y_train_pred)
    test_score = accuracy_score(y_test, y_test_pred)
    print("\n\n------ Lambda = {} ------".format(lmdb))
    print('Train accuracy score = {:.3f}'.format(train_score))
    print('Test accuracy score = {:.3f}\n\n'.format(test_score))
    res.append([lmdb,test_score])


			######## y has been reformated to {-1,1} ########



/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  4e+03  6e+01  1e+00
 1: -3.5885e+00 -4.2893e+00  1e+02  2e+00  3e-02
 2: -2.0624e+00 -2.2143e+00  2e+01  3e-01  4e-03
 3: -8.7137e-01 -1.7635e+00  2e+00  2e-02  3e-04
 4: -7.9391e-01 -1.0971e+00  3e-01  5e-18  5e-15
 5: -8.9266e-01 -9.2526e-01  3e-02  5e-18  5e-15
 6: -9.0552e-01 -9.0783e-01  2e-03  5e-18  6e-15
 7: -9.0641e-01 -9.0683e-01  4e-04  5e-18  5e-15
 8: -9.0656e-01 -9.0667e-01  1e-04  5e-18  5e-15
 9: -9.0661e-01 -9.0661e-01  3e-06  5e-18  6e-15
10: -9.0661e-01 -9.0661e-01  3e-08  5e-18  6e-15
Optimal solution found.


------ Lambda = 0.001 ------
Train accuracy score = 0.743
Test accuracy score = 0.610



			######## y has been reformated to {-1,1} ########

     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  4e+03  6e+01  1e+00
 1: -2.5480e+01 -1.6692e+01  5e+02  7e+00  1e-01
 2: -9.1937e+00 -3.7764e+00  9e+01  1e+00  2e-02
 3: -1.8851e+00 -1.9509e+00  1e+01  1e-01  2e-03


In [56]:
best_pred = sorted(res, key = lambda x:x[1])[-1]
print(best_pred)

[1000, 0.615]


In [27]:
print(lmbd)

100000


In [50]:
lmbd = 100000
model = SVM(lmbd=lmbd)
model.train(K_train, y_train)
y_train_pred = model.predict(K_train)
y_test_pred = model.predict(K_test)
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)
print("\n\n------ Lambda = {} ------".format(lmbd))
print('Train accuracy score = {:.3f}'.format(train_score))
print('Test accuracy score = {:.3f}\n\n'.format(test_score))


			######## y has been reformated to {-1,1} ########



/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  3e+03  6e+01  1e+00
 1: -9.9134e-01 -1.7627e+00  4e+01  6e-01  1e-02
 2: -7.1732e-01 -1.6974e+00  1e+00  6e-17  2e-15
 3: -9.2718e-01 -1.0070e+00  8e-02  7e-18  2e-15
 4: -9.9927e-01 -1.0001e+00  9e-04  5e-18  2e-15
 5: -9.9999e-01 -1.0000e+00  9e-06  6e-18  2e-15
 6: -1.0000e+00 -1.0000e+00  9e-08  5e-18  2e-15
Optimal solution found.


------ Lambda = 100000 ------
Train accuracy score = 0.772
Test accuracy score = 0.565




In [53]:
lmbd = 5e-4
model = SVM(lmbd=lmbd)
model.train(K_train, y_train)
y_train_pred = model.predict(K_train)
y_test_pred = model.predict(K_test)
train_score = accuracy_score(y_train, y_train_pred)
test_score = accuracy_score(y_test, y_test_pred)
print("\n\n------ Lambda = {} ------".format(lmbd))
print('Train accuracy score = {:.3f}'.format(train_score))
print('Test accuracy score = {:.3f}\n\n'.format(test_score))


			######## y has been reformated to {-1,1} ########



/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2111: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'x' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2116: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 's' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2131: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'y' in initvals:
/Users/karlhajjar/anaconda/lib/python3.5/site-packages/cvxopt/coneprog.py:2136: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if 'z' in initvals:


     pcost       dcost       gap    pres   dres
 0:  0.0000e+00 -1.0000e+00  3e+03  6e+01  1e+00
 1: -2.4207e+00 -3.2786e+00  1e+02  2e+00  3e-02
 2: -1.5022e+00 -2.0293e+00  1e+01  2e-01  4e-03
 3: -7.3850e-01 -1.6413e+00  2e+00  1e-02  2e-04
 4: -7.2831e-01 -9.8523e-01  3e-01  2e-03  3e-05
 5: -7.7176e-01 -8.6485e-01  1e-01  5e-04  8e-06
 6: -7.9458e-01 -8.1709e-01  2e-02  6e-05  1e-06
 7: -8.0247e-01 -8.0420e-01  2e-03  1e-07  2e-09
 8: -8.0322e-01 -8.0332e-01  1e-04  7e-09  1e-10
 9: -8.0326e-01 -8.0326e-01  3e-06  2e-10  3e-12
10: -8.0326e-01 -8.0326e-01  7e-08  2e-12  4e-14
Optimal solution found.


------ Lambda = 0.0005 ------
Train accuracy score = 0.787
Test accuracy score = 0.570




In [39]:
X_train

array([[3, 0, 1, ..., 2, 0, 3],
       [0, 0, 1, ..., 2, 0, 1],
       [3, 0, 3, ..., 0, 3, 1],
       ...,
       [3, 1, 1, ..., 3, 2, 1],
       [0, 3, 2, ..., 3, 0, 0],
       [1, 1, 3, ..., 3, 1, 0]])

In [40]:
X_test

array([[3, 3, 2, ..., 1, 3, 2],
       [1, 0, 0, ..., 0, 0, 2],
       [0, 2, 0, ..., 3, 3, 1],
       ...,
       [3, 0, 2, ..., 1, 0, 1],
       [3, 1, 2, ..., 3, 3, 3],
       [2, 1, 3, ..., 1, 0, 2]])

In [16]:
def transform(y, threshold=0.5):
    res = np.zeros(len(y))
    ones = y >= threshold
    res[ones] = 1
    res[~ones] = 0
    return res

In [26]:
res = []
for lmdb in [1e-5, 1e-4, 0.001,0.01,0.05]:
    print("------ Lambda = {} ------".format(lmdb))
    model = KRR(lmbd=lmdb)
    model.train(K_train, y_train)
    y_train_pred = model.predict(K_train)
    y_test_pred = model.predict(K_test)
    for threshold in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8]:
        print('\t\tthreshold = {}'.format(threshold))
        res_train = transform(y_train_pred, threshold=threshold)
        res_test = transform(y_test_pred, threshold=threshold)
        train_score = accuracy_score(y_train, res_train)
        test_score = accuracy_score(y_test, res_test)
        print('Train accuracy score = {:.3f}'.format(train_score))
        print('Test accuracy score = {:.3f}\n'.format(test_score))
        res.append([lmdb, threshold, test_score])

------ Lambda = 1e-05 ------
		threshold = 0.1
Train accuracy score = 0.514
Test accuracy score = 0.497

		threshold = 0.2
Train accuracy score = 0.526
Test accuracy score = 0.515

		threshold = 0.3
Train accuracy score = 0.544
Test accuracy score = 0.530

		threshold = 0.4
Train accuracy score = 0.568
Test accuracy score = 0.535

		threshold = 0.5
Train accuracy score = 0.589
Test accuracy score = 0.540

		threshold = 0.6
Train accuracy score = 0.569
Test accuracy score = 0.510

		threshold = 0.7
Train accuracy score = 0.538
Test accuracy score = 0.490

		threshold = 0.8
Train accuracy score = 0.524
Test accuracy score = 0.490

------ Lambda = 0.0001 ------
		threshold = 0.1
Train accuracy score = 0.514
Test accuracy score = 0.495

		threshold = 0.2
Train accuracy score = 0.525
Test accuracy score = 0.515

		threshold = 0.3
Train accuracy score = 0.543
Test accuracy score = 0.530

		threshold = 0.4
Train accuracy score = 0.568
Test accuracy score = 0.535

		threshold = 0.5
Train accur

In [30]:
best_pred = sorted(res, key = lambda x:x[2])[-1]
print(best_pred)

[0.05, 0.4, 0.5525]
